In [2]:
import pandas as pd
import DB as db
import numpy as np

# test 1

In [3]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2022-12-11.zip"
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-20.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-19.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-18.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False).query("token == 'confirmLandReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])
# logdf.head()

In [ ]:
# logdf.query("Land_ID in (80312, 80487)")
logdf.query("NID == '26312132702012'")

In [ ]:
land_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\DB-summary\DB checksum\NewQueryLand.zip"
land_app_fn =r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\checksum\NewQueryLand.zip"
dbdf= pd.read_csv(land_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 86")
    # .query("project_id == 86 and land_application_status == 'W'")[['applicant_national_id', 'check_sum', 'land_id', 'project_id']]

In [ ]:
dbdf.query("land_id == '80487'")

In [ ]:
df = (dbdf.merge(logdf, left_on='applicant_national_id', right_on='NID')
      .assign(match = lambda x:(x.checksum == x.check_sum))
    #   .fillna('land_id', axis=1)
    #   .assign(land_id = lambda x: x.astype(np.int64) )
    )
# df.info()

In [ ]:
df.match.value_counts()

In [ ]:
df.query("match == False").assign(land_id = lambda x: x.land_id.astype(np.int64)).to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

## UNITS

In [10]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-02-07.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False, dtype={'NID':str}).query("token == 'confirmReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])



In [11]:
logdf.query("NID == '26411200102736'")

,log_date,line_no,NID,log_type,country,IP_address,service,token,categ,Gov,City,Region,District,Sub_District,Floor_No,building_no,Unit_No,Unit_Model,Unit_ID,checksum
42650,2023-02-07 10:32:58,6012965,26411200102736,ERROR,EG,197.162.113.115,confirmReservation,confirmReservation True,user,القاهرة,القاهرة الجديدة,نزهة الأندلس عمارات مدخل (أ),NaN,NaN,0.0,15.0,17.0,-,157427.0,0141997DCF07912E00BFFCD7637391BB3E6EDC64E6D1FB...


In [ ]:
logdf.token.value_counts()
logdf.to_csv('./out/mis_log.csv')

In [4]:
unit_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\checksum\checksum.zip"
dbdf= pd.read_csv(unit_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 87")

In [5]:
dbdf.query("applicant_national_id == '26411200102736'")

,project_id,applicant_national_id,check_sum,land_id
37689,87,26411200102736,0141997DCF07912E00BFFCD7637391BB3E6EDC64E6D1FB...,157427


In [ ]:
dbdf.project_id.value_counts().sort_index(ascending=False)

In [ ]:
dbdf.any()

In [ ]:
df = (dbdf.merge(logdf, how='outer', left_on='applicant_national_id', right_on='NID')
      # .assign(match = lambda x:(x.checksum == x.check_sum))
)


In [ ]:
df.to_csv('./out/checksum.csv')

In [ ]:
df.match.value_counts()

In [ ]:
df.query("match == False").assign(unit_id = lambda x: x.unit_id.astype(np.int64))[['NID', 'check_sum', 'checksum']]
#.to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

# LE04

In [ ]:
le04 = pd.read_excel(r"C:\Users\yahia\Downloads\le04.xls", skiprows=1)

In [ ]:
le04.iloc[:,[0,1,2,3,17]].rename({
    'نسبة التميز': 'excellence_ratio',
    'مساحة الأرض':'land_size',
    'رقم الأرض':'Land_No',
    'كود المجاورة':'Sub_District',
    'الرقم القومى':'NID'
}, axis=1)


### new zip file

In [46]:
projdf = (db.query_to_pd("project")
        .set_index('project_id')
        .drop(columns=['index','Unnamed: 3','Unnamed: 11','Unnamed: 13'], axis=1)
        .assign (start_date = lambda x: x.start_date.str[:10])
        .assign (end_date= lambda x: x.end_date.str[:10])
        .assign (publish_date= lambda x: x.publish_date.str[:10])
        .rename(columns={'project_type_name_ar':'proj_type', 'No. of reservations':'n_Resv'})
        .assign(n_Resv = lambda x: x.fillna(0).n_Resv.astype(int))
        .assign(_ = lambda x: x.insert(0,'select', False) )
        .sort_values(by='start_date', ascending=False)
    )
# projdf.insert(0,'select', False)
# projdf['# Res.'].fillna(0).astype(int)
projdf.head(4)

,select,proj_type,project_name_ar,publish_date,start_date,end_date,publish_end_date,No. of units/lands,No. of applications,No. of paid customers,n_Resv,selelct
project_id,,,,,,,,,,,,
89,False,وحدات سكنية,مشروع الجوهرة بمدينة بورسعيد,2023-06-11,2023-07-04,2023-07-04,2023-07-04 00:00:00,20,260,42,20,None
88,False,وحدات سكنية,مشروع الفيروز بمدينة الغردقة مرحلة ثالثة,2023-06-11,2023-07-03,2023-07-03,2023-07-03 00:00:00,15,795,185,15,None
86,False,أراضى,قطع أراضي إسكان متوسط محور الاراضي مرحلة ثالثه,2023-01-15,2023-04-18,2023-04-20,2023-04-20 00:00:00,7322,36450,27482,7321,None
85,False,أراضى,قطع أراضي مميزة مرحلة الحاديه عشره,2023-01-15,2023-04-13,2023-04-13,2023-04-13 00:00:00,3481,3358,1499,1187,None


,index,proj_type,project_name_ar,Unnamed: 3,publish_date,start_date,end_date,publish_end_date,No. of units/lands,No. of applications,No. of paid customers,Unnamed: 11,# Res.,Unnamed: 13,select
project_id,,,,,,,,,,,,,,,
89,0,وحدات سكنية,مشروع الجوهرة بمدينة بورسعيد,None,2023-06-11,2023-07-04,2023-07-04,2023-07-04 00:00:00,20,260,42,0,20.0,None,False
88,1,وحدات سكنية,مشروع الفيروز بمدينة الغردقة مرحلة ثالثة,None,2023-06-11,2023-07-03,2023-07-03,2023-07-03 00:00:00,15,795,185,0,15.0,None,False
86,3,أراضى,قطع أراضي إسكان متوسط محور الاراضي مرحلة ثالثه,None,2023-01-15,2023-04-18,2023-04-20,2023-04-20 00:00:00,7322,36450,27482,0,7321.0,None,False
85,4,أراضى,قطع أراضي مميزة مرحلة الحاديه عشره,None,2023-01-15,2023-04-13,2023-04-13,2023-04-13 00:00:00,3481,3358,1499,0,1187.0,None,False
84,5,أراضى,قطع أراضي أكثر تميزا مرحلة عاشره,None,2023-01-15,2023-04-12,2023-04-12,2023-04-12 00:00:00,425,5272,2503,0,274.0,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,76,وحدات سكنية,مشروع كومباوند اشراقة - مدينة 6 اكتوبر/ مشروع...,None,2019-02-04,2019-02-24,2019-03-10,2019-03-10 00:00:00,69,162,5,0,5.0,None,False
3,79,وحدات سكنية,مدينتى - للعاملين بالوزارة و الهيئة و الأجهزة...,None,2018-12-23,2019-02-17,2019-03-07,2019-03-07 00:00:00,1000,2099,1532,0,1000.0,None,False
4,78,وحدات سكنية,سكن مصر - المرحلة التكميلية,None,2019-01-06,2019-02-03,2019-02-14,2019-02-14 00:00:00,5694,1469,832,0,724.0,None,False
